# MACHINE LEARNING DO ZERO EM PYTHON

# 0. INTRODUÇÃO

BREVE APRESENTAÇÃO: <br>
- Graduado em Matemática pela USP, pós-graduado em Finanças pela FIA e mestre em Economia pela FGV;
- Passagem por empresas como Itaú Unibanco, HSBC e Vivo. Atualmente, cientista de dados na startup de serviços, GetNinjas;
- Experiência com riscos, cobrança, crédito e precificação;
- Blog de Data Science: www.EstatSite.com.br;
- Canal do Yukio (Youtube): https://www.youtube.com/channel/UCZDVnGEyggjuo2kgpmXdzGA;
- Twitter: @EstatSite;
- Podcast de Tecnologia: Pitacotech;
- Github: www.github.com/yukioandre;

O QUE É MACHINE LEARNING? <br>
<img src='AI-ML-DS.png' style="width: 300px">
- Como o próprio nome já diz, estamos criando máquinas que aprendam (no caso, com dados);
- Exemplo 1: Um filtro de spam que é capaz de identificar caracteres "estranhos" e que sejam indícios de que aquele e-mail é um spam é um programa de aprendizado de máquinas. Exemplo 2: Um programa que seja capaz de identificar se uma avaliação está elogiando ou insultando o estabelecimento também é um aprendizado de máquinas. Por isso aprendizado de máquinas é um campo da Inteligência Artificial - estamos tratando de máquinas que aprendem.
- Polêmica: Por mais que ainda exista relutância de algumas pessoas, machine learning não é estatística/matemática com um nome mais chique;
- Machine learning é sobre predições, pouco esforço humano e aprendizado com os dados;
- Cases famosos: modelo de predição de assintomáticos com Covid; competição para criação do modelo de recomendação da Netflix; predição de Alzheimer.
- Principais tipos de aprendizado: Supervisionado e Não-Supervisionado.

O MERCADO DE MACHINE LEARNING: <br>
- Muito recente, em alto crescimento;
- Alta remuneração (não aquelas dos jornais, mas ainda alta se comparada com a média de outras áreas);
- Áreas e cargos ainda em construção, em constante mudança;
- Foco em aprendizado constante: 90% dos cientistas de dados  (Coursera, Datacamp, Udacity, Alura, etc).

## 1. CARREGA BIBLIOTECAS

In [1]:
# Para manipulacao dos dados
import pandas as pd
import numpy as np

# para graficos
import seaborn as sns
import matplotlib.pyplot as plt
import sweetviz as sv # !pip install sweetviz

# para modelagem
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid, KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


#### Settings ###########
pd.set_option("max_colwidth", 1000)
pd.set_option("max_rows", 20)
pd.set_option("max_columns", 1000)
pd.set_option("precision", 2)
pd.options.display.float_format = "{:,.2f}".format
plt.style.use("classic")
%load_ext nb_black

OptionError: Pattern matched multiple keys

## 2. PRIMEIRAS IMPRESSÕES

Mais sobre os campos do dataset:
- Age (numeric)
- Sex (text: male, female)
- Job (numeric: 0 — unskilled and non-resident, 1 — unskilled and resident, 2 — skilled, 3 — highly skilled)
- Housing (text: own, rent, or free)
- Saving accounts (text — little, moderate, quite rich, rich)
- Checking account (numeric, in DM — Deutsch Mark)
- Credit amount (numeric, in DM)
- Duration (numeric, in month)
- Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

In [ ]:
# carrega dataset
df = pd.read_csv("german_credit_data.csv", index_col=0)  # sep = ';', decimal = ','

In [ ]:
# ve as primeiras linhas
df.head()

In [ ]:
# ve as ultimas linhas
df.tail()

In [ ]:
df.head(10)

In [ ]:
# informação das variáveis (tipo, valores nulos, memoria que o objeto ta consumindo, etc)
df.info()

In [ ]:
df.shape

In [ ]:
# f-string
print(f"O dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas.")

In [ ]:
# Como acessar os valores de shape:
df.shape[1]

In [ ]:
# usar f-string dentro de queries
import pandasql as ps

filtro_idade = 70
query = f"""
select * from df
where age < {filtro_idade}
"""

ps.sqldf(query, locals())

In [ ]:
# Acessar elementos: loc e o iloc
df.iloc[0:4, 0:7]  # repara: está acessando um intervalo aberto na direita

In [ ]:
# outro modo de acessar elementos, usando nome da coluna
df.loc[0:3, "Age"]

In [ ]:
# se quiser trazeer mais de uma coluna
df.loc[0:3, ["Age", "Sex", "Job"]]

In [ ]:
# também pode ir pelo nome da coluna como índice
novo_dataframe = df[["Age", "Housing"]]
novo_dataframe.head()

In [ ]:
# Estatística descritiva
df.describe()

In [ ]:
# E se quiser a estatística descritiva por grupo?
df.groupby("Sex")["Age", "Credit amount", "Duration"].mean()

In [ ]:
df.groupby("Purpose")["Age", "Credit amount", "Duration"].mean()

In [ ]:
# % de missing
df.isna().mean() # se quisesse preencher, poderia usar fillna() 

# (ver post 'Classificador de Estilo Musical' lá no blog)

In [ ]:
# total de missing por variavel
df.isna().sum()

In [ ]:
# olhando as variáveis categóricas
df.Housing.value_counts()

In [ ]:
df.Sex.value_counts()

In [ ]:
# ver o percentual:
df.Housing.value_counts(normalize=True).sort_index()

In [ ]:
# pegando os nomes das colunas
df.columns

In [ ]:
# Filtros
# Query para lógica
# 1)
df.query("Age > 70")

In [ ]:
# 2)
homens_velhos = df.query(" Age > 70 & Sex == 'male' ")
homens_velhos.head()

In [ ]:
# Método para tentar inferir se a variável é categórica
provavel_categorica = {}

for var in df.columns:
    provavel_categorica[var] = 1.0 * df[var].nunique() / df[var].count() < 0.03

provavel_categorica

In [ ]:
# relembrando
df.head()

In [ ]:
df["Target"] = np.where(df["Risk"] == "bad", 1, 0)
df.head()

In [ ]:
df.drop("Risk", axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.Target.value_counts(normalize=True)

## 3. ANÁLISE EXPLORATÓRIA

PLOT COM PANDAS

In [ ]:
_ = plt.figure(figsize=(8, 4))
_ = df.Target.value_counts().plot(kind="bar")
_ = plt.title("Nº de Clientes por Tipo de Risco")
_ = plt.ylim(0, 750)

In [ ]:
_ = df.groupby("Target")["Age"].mean().plot(kind="bar")
_ = plt.title("Idade Média por Tipo de Risco")

PLOTS COM MATPLOTLIB (FOCO NA HIERARQUIA)

<img src='matplotlib_framework.png' style="width: 500px">

In [ ]:
# Veja como o gráfico vai sendo construído por partes
_ = plt.figure(figsize=(6, 4))
_ = plt.xlabel("Idade")
_ = plt.ylabel("Quantidade")

In [ ]:
# Note que vamos adicionando elementos ao desenho
_ = plt.figure(figsize=(7, 4))
_ = plt.hist(data=df, x="Age", bins=20, rwidth=0.9, color="red")
_ = plt.xlabel("Idade")
_ = plt.title("Histograma para Idade")

PLOT COM SEABORN

In [ ]:
_ = sns.relplot(
    x="Duration",
    y="Credit amount",
    hue="Target",
    palette=["purple", "blue"],
    size="Age",
    data=df,
)

In [ ]:
# Gráfico de contagem por categoria
_ = sns.catplot(x="Housing", kind="count", data=df)

DASHES AUTOMATIZADOS

In [ ]:
# Dashboard com UMA única linha
reporte = sv.analyze(df)
reporte.show_html()

## 4. MODELAGEM DE MACHINE LEARNING: RANDOM FOREST

Primeiro, um exemplo simples de árvore de decisão:

<img src='arvore_de_decisao.png'>

Mais sobre o modelo, você encontra aqui: https://estatsite.com.br/2016/06/11/1970/

Agora, vejamos a Random Forest, um algoritmo baseado em árvores de decisão:

<img src='Random_forest_diagram_complete.png'>

RF combina a simplicidade das árvores de decisão com a flexibilidade de aprender de amostras novas (menor risco de overfitting).

Como funciona a Random Forest:
- Coletamos uma amostra do dataset original, podendo haver repetições;
- Começamos a construção de uma árvore de decisão a partir do dataset gerado pela amostragem acima;
- Na hora de selecionar qual a feature do primeiro nó (=root node), consideramos um subconjunto das features disponíveis no dataset;
- Na hora de escolher a feature do nó seguinte, também selecionamos a partir de um subconjunto das variáveis que restaram;
- Seguimos fazendo isso até finalizar a árvore;
- Note: o subconjunto de features pode ter tamanho 2, 3, ..., n. É preciso escolher o que gera melhor desempenho;
- Pegue o processo e repita centenas de vezes. I.e., construímos centenas de árvores de decisão.
- Como usamos as árvores?
- Pegue o primeiro data point (o primeiro "indivíduo" do nosso conjunto de dados) e rode ele na primeira árvore. Suponha que a gente esteja construindo um modelo para previsão de bom ou mau pagador. A previsão da primeira árvore é que o primeiro indivíduo é um mau pagador. Aí, rodamos para a segunda árvore. Ela também diz que ele será um bom pagador. Repetimos isso para todas árvores. Vemos qual opção recebeu mais votos. Como a maioria das árvores teve como previsão que o indivíduo seria um bom pagador, nossa previsão é que ele será um bom pagador.

Termo importante:
- Ensemble learning: Uso de múltiplos algoritmos para obter melhor desempenho preditivo.

In [ ]:
# Vamos listar as features que vamos utilizar
features = ["Age", "Job", "Credit amount", "Purpose", "Housing", "Duration"]
target = "Target"

In [ ]:
# Agora, dividimos antes de qualquer pré-processamento
X = df[features]  # Features
y = df[target]  # Labels

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=999
)

In [ ]:
# Precisamos tratar as variáveis categóricas (Cuidado quando for numérica)
cat_features = ["Job", "Housing", "Purpose"]
num_features = ["Age", "Credit amount", "Duration"]

In [ ]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=cat_features)

X_train = encoder.fit_transform(X_train)

X_train.head()

In [ ]:
X_test = encoder.transform(X_test)

X_test.head()

In [ ]:
clf_RF = RandomForestClassifier()  # instanciar (= inicializar, criar o objeto)
clf_RF.fit(X_train, y_train)  # treina o modelo

In [ ]:
# aplica no teste
y_pred = clf_RF.predict(X_test)

y_pred

In [ ]:
# metricas de avaliacao
from sklearn.metrics import accuracy_score

print("Acurácia: " + str(accuracy_score(y_test, y_pred)))  # acertos ao todo

from sklearn.metrics import f1_score

print(
    "F1 Score: {}".format(f1_score(y_test, y_pred))
)  # indica poucos falsos positivos e falsos negativos, quanto mais próximo de 1, melhor

In [ ]:
from sklearn.metrics import confusion_matrix

print("Matriz de Confusão : \n" + str(confusion_matrix(y_test, y_pred)))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [ ]:
print("True Positive: " + str(tp))
print("True Negative: " + str(tn))
print("False Positive: " + str(fp))
print("False Negative: " + str(fn))

In [ ]:
21 / (21 + 27)

In [ ]:
# CURVA ROC: calcula fpr e tpr para vários limiares
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_predict

# probabilidades
# probs = clf_RF.predict_proba(X_test)

y_scores = cross_val_predict(clf_RF, X_test, y_test)

# obtem fpr, tpr e limites
fpr, tpr, thresholds = roc_curve(y_test, y_scores)


def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], "k--")
    plt.axis([0, 1, 0, 1])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")


plot_roc_curve(fpr, tpr)

In [ ]:
clf_RF.feature_importances_

In [ ]:
# FEATURE IMPORTANCE
feature_imp = pd.Series(clf_RF.feature_importances_, index=X_train.columns).sort_values(
    ascending=False
)
feature_imp

In [ ]:
# Creating a bar plot
_ = plt.figure(figsize=(10, 6))
_ = sns.barplot(x=feature_imp, y=feature_imp.index)

# Add labels to your graph
_ = plt.xlabel("Feature Importance Score")
_ = plt.ylabel("Features")
_ = plt.title("Visualizing Important Features")
_ = plt.savefig("rf_features.png")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, truncnorm, randint

# Tunando os hiperparâmetros:
model_params = {
    # randomly sample numbers from 4 to 204 estimators
    "n_estimators": randint(4, 200),
    # normally distributed max_features, with mean .25 stddev 0.1, bounded between 0 and 1
    "max_features": truncnorm(a=0, b=1, loc=0.25, scale=0.1),
    # uniform distribution from 0.01 to 0.2 (0.01 + 0.199)
    "min_samples_split": uniform(0.01, 0.199),
}

# create random forest classifier model
rf_model = RandomForestClassifier()

# set up random search meta-estimator
# this will train 100 models over 5 folds of cross validation (500 models total)
clf = RandomizedSearchCV(rf_model, model_params, n_iter=10, cv=5, random_state=1)

# train the random search meta-estimator to find the best model out of 100 candidates
model = clf.fit(X_train, y_train)

# print winning set of hyperparameters
from pprint import pprint

pprint(model.best_estimator_.get_params())

In [ ]:
# Qual combinação de parâmetros trouxe melhor resultado:
model.best_estimator_

In [ ]:
clf_random = RandomForestClassifier(
    max_features=0.3124639258611636,
    min_samples_split=0.05068599769657197,
    n_estimators=160,
)

clf_random.fit(X_train, y_train)

In [ ]:
y_pred_random = clf_random.predict(X_test)

print("Acurácia: " + str(accuracy_score(y_test, y_pred_random)))

print("F1 Score: {}".format(f1_score(y_test, y_pred_random)))